<a href="https://colab.research.google.com/github/peremartra/Rearchitecting-LLMs/blob/main/CH05/CH05_NB02_data_sms_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rearchitecting LLMs
## Surgical Optimization for Hyper-Efficient Models


### Chapter 5: Width Pruning
### Notebook: 02. Data-Driven Neuron Selection
by [Pere Martra](https://github.com/peremartra)

[![LinkedIn](https://img.shields.io/badge/LinkedIn-0077B5?style=flat&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/pere-martra/) [![GitHub](https://img.shields.io/badge/GitHub-100000?style=flat&logo=github&logoColor=white)](https://github.com/peremartra) [![X](https://img.shields.io/badge/X-000000?style=flat&logo=x&logoColor=white)](https://x.com/PereMartra) [![Hugging Face](https://img.shields.io/badge/🤗%20Hugging%20Face-blue)](https://huggingface.co/oopere)

_____
Colab Environment: GPU T4

Models:
* Llama-3.2-1B
_____

In this notebook, we explore **data-driven width pruning**—a more sophisticated approach to neural network compression that uses activation patterns from real data to make pruning decisions. Unlike the static magnitude-based pruning from Notebook 01, this method captures how neurons actually behave during inference.

We implement a **CFSP-inspired hybrid importance scoring** that combines:
- **Static component**: Weight magnitudes (what neurons *could* do)
- **Dynamic component**: Activation norms from calibration data (what neurons *actually* do)

The key experiment: We create two differently-pruned models from the same Llama-3.2-1B base, using contrasting calibration datasets:
- **WikiText-2**: Long-form encyclopedic text with complex language patterns
- **SMS Spam**: Short conversational messages with informal language

By cross-evaluating all three models (base, wiki-pruned, sms-pruned) on both datasets, we answer a critical question: **Does the choice of calibration data influence which neurons get pruned, and does this create domain-specialized models?**

By the end of this notebook, you'll understand how activation-aware pruning differs from static methods, why calibration dataset selection matters, and whether domain-specific pruning offers practical advantages over one-size-fits-all compression.

# Setting up notebook

In [1]:
!pip install -q \
      "torch" \
      "transformers==4.55.4" \
      "accelerate==1.10.1" \
      "lm_eval==0.4.9.1" \
      "sentencepiece==0.2.1" \
      "datasets" \
      "langdetect"\
      "optipfair==0.2.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 66.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from lm_eval import evaluator
from torch import nn
from lm_eval.models.huggingface import HFLM
import os
import json
import copy
import gc
import time
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU: Tesla T4


Download helper functions from the repository.

In [4]:
# Download utils.py from GitHub repository
!wget -q https://raw.githubusercontent.com/peremartra/Rearchitecting-LLMs/main/utils.py

# Verify download
import os
if os.path.exists('utils.py'):
    print("✅ utils.py downloaded successfully")
else:
    print("❌ Failed to download utils.py")

from utils import (
  model_evaluation, # Evals with lm_eval
  evaluate_metrics, # Loss & Perpelexity
  generate_text, #test inference model
  clear_gpu_cache
)

✅ utils.py downloaded successfully


In [5]:
def measure_detailed_performance(model, tokenizer, data_source, num_runs=3, max_new_tokens=50, max_samples=None):
    """
    Measures inference performance metrics.

    Args:
        model: Model to evaluate
        tokenizer: Tokenizer
        data_source: DataLoader to sample from
        num_runs: Number of runs per sample for averaging
        max_new_tokens: Tokens to generate
        max_samples: Limit number of samples (None = all)

    Returns:
        dict with timing statistics
    """
    model.eval()
    times = []
    tokens_generated = []

    samples = []
    for batch in data_source:
        for i in range(len(batch['input_ids'])):
            samples.append(batch['input_ids'][i])
            if max_samples and len(samples) >= max_samples:
                break
        if max_samples and len(samples) >= max_samples:
            break

    if max_samples:
        samples = samples[:max_samples]

    print(f"Measuring performance on {len(samples)} samples...")

    with torch.no_grad():
        for sample in tqdm(samples, desc="Performance test"):
            input_ids = sample.unsqueeze(0).to(device)

            for _ in range(num_runs):
                start_time = time.time()
                outputs = model.generate(
                    input_ids,
                    max_new_tokens=max_new_tokens,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id
                )
                end_time = time.time()

                elapsed = end_time - start_time
                times.append(elapsed)
                tokens_generated.append(outputs.shape[1] - input_ids.shape[1])

    avg_time = np.mean(times)
    std_time = np.std(times)
    avg_tokens = np.mean(tokens_generated)
    tokens_per_sec = avg_tokens / avg_time if avg_time > 0 else 0

    return {
        'avg_time_sec': avg_time,
        'std_time_sec': std_time,
        'avg_tokens': avg_tokens,
        'tokens_per_sec': tokens_per_sec,
        'num_samples': len(samples),
        'num_runs': num_runs
    }

## Configuration Parameters

In [6]:
# Model configuration
MODEL_NAME = 'meta-llama/Llama-3.2-1B'

# Dataset configuration
RECOVERY_SAMPLES = 100  # Calibration samples per dataset
MAX_LENGTH = 1024
BATCH_SIZE = 4

# Pruning configuration
PRUNE_PERCENT = 0.2  # 20% of neurons will be pruned

# Generation configuration
MAX_NEW_TOKENS = 50



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()
model.generation_config.temperature = None
model.generation_config.top_p = None
model.generation_config.top_k = None

print(f"✓ Loaded {MODEL_NAME}")
print(f"  Layers: {len(model.model.layers)}")
print(f"  Hidden size: {model.config.hidden_size}")
print(f"  Intermediate size: {model.config.intermediate_size}")

In [8]:
# Test the original model
prompt = "Paris is the capital of"
generated_base = generate_text(model, tokenizer, prompt, device)
print(f"Base model generation: {generated_base}")

Base model generation: Paris is the capital of France and the largest city in the country. It is located on the River Seine and is one of the most popular tourist destinations in Europe. The city has a rich history and culture, and it is home to many famous landmarks, including the E


# Load and Prepare Calibration Datasets

We'll load two contrasting datasets to serve as calibration sources for our pruning experiments:

1. **WikiText-2**: Long-form, encyclopedic text with complex language patterns. This dataset represents formal, structured writing with varied vocabulary and sophisticated grammar. It's designed to test a model's ability to handle coherent, knowledge-rich narratives.

2. **SMS Spam**: Short conversational messages with informal language. This dataset contains brief text messages—often with abbreviations, casual grammar, and simple sentence structures. It represents a completely different linguistic domain.

**The Hypothesis**: Different datasets will activate different neurons during calibration. When we calculate importance scores based on actual activation patterns, neurons that fire strongly on WikiText might barely activate on SMS, and vice versa. This should produce two distinct pruning patterns—and potentially, two specialized models.

These datasets will serve as calibration sources for our two pruned models, allowing us to test whether domain-specific calibration creates measurably different architectures.

In [ ]:
# Load datasets
datawiki = load_dataset('wikitext', 'wikitext-2-raw-v1', split=f'train[:{RECOVERY_SAMPLES}]')
datasms = load_dataset('sms_spam', split=f'train[:{RECOVERY_SAMPLES}]')

print(f"✓ WikiText samples: {len(datawiki)}")
print(f"✓ SMS samples: {len(datasms)}")

In [10]:
def prepare_dataset(dataset, text_field='text'):
    """
    Tokenizes and prepares a dataset for calibration.

    Handles different dataset formats (WikiText uses 'text', SMS uses 'sms' field).
    """
    def tokenize_function(examples):
        if text_field in examples:
            texts = examples[text_field]
        elif 'sms' in examples:  # SMS dataset specific
            texts = examples['sms']
        elif 'text' in examples:
            texts = examples['text']
        else:
            texts = examples[list(examples.keys())[0]]

        return tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=MAX_LENGTH,
            return_tensors='pt'
        )

    tokenized = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
    tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    return DataLoader(tokenized, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Create dataloaders
dataloaderwiki = prepare_dataset(datawiki)
dataloadersms = prepare_dataset(datasms)

print(f"✓ Created dataloaders")
print(f"  Wiki batches: {len(dataloaderwiki)}")
print(f"  SMS batches: {len(dataloadersms)}")

In [12]:
metrics_base_wiki = evaluate_metrics(model, dataloaderwiki)


Evaluating: 100%|██████████| 25/25 [00:14<00:00,  1.71it/s]


In [13]:
metrics_base_sms = evaluate_metrics(model, dataloadersms)


Evaluating: 100%|██████████| 25/25 [00:14<00:00,  1.69it/s]


In [14]:
base_wiki_timing = measure_detailed_performance(model, tokenizer, dataloaderwiki, max_samples=10)

Measuring performance on 10 samples...


Performance test: 100%|██████████| 10/10 [00:20<00:00,  2.04s/it]


In [15]:
base_sms_timing = measure_detailed_performance(model, tokenizer, dataloadersms, max_samples=10)

Measuring performance on 10 samples...


Performance test: 100%|██████████| 10/10 [00:32<00:00,  3.29s/it]


In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())


In [17]:
original_params = count_parameters(model)

In [18]:
clear_gpu_cache()


# Data-Driven Pruning Functions

These functions implement the CFSP-inspired methodology adapted from contextual pruning research. The key innovation: instead of pruning based solely on weight magnitudes (static), we incorporate **runtime activation patterns** (dynamic) to make smarter pruning decisions.

## Understanding the Approach

**Static pruning** (Notebook 01) asks: "Which neurons have the smallest weights?"  
**Data-driven pruning** (this notebook) asks: "Which neurons have the smallest weights *and* barely activate on real data?"

This is accomplished through three stages:
1. **Activation capture**: PyTorch hooks on down_proj to record runtime behavior
2. **Hybrid importance**: Combines weight magnitudes with activation norms  
3. **Neuron pair pruning**: Removes least important neurons from gate_proj, up_proj, and down_proj

**Why down_proj?** In the GLU MLP architecture, down_proj receives the output of the gating mechanism:

```
X → gate_proj → SiLU(·) ──┐
                          × → down_proj → output
X → up_proj ──────────────┘
```

The input to down_proj (X_d) represents the **post-gating activations**—the actual information that flows through each neuron after the gate decides what to pass. By measuring ||X_d||, we capture which neurons contribute meaningful activations during forward passes on our calibration data.

## Activation Capture with PyTorch Hooks

**What are PyTorch hooks?** Hooks are callback functions that PyTorch executes automatically during forward or backward passes. They let us "spy" on intermediate activations without modifying the model's code.

In our case, we register **forward hooks** on the `down_proj` layer of every MLP module. When data flows through the model during calibration, our hook function captures the input tensor to `down_proj` (X_d), calculates its L2 norm, and accumulates these norms across all batches.

**Why accumulate?** A single batch might not represent typical activation patterns. By summing L2 norms across all calibration batches, we get a robust estimate of each neuron's average contribution. Neurons with consistently high ||X_d|| values are considered important and preserved; those with low norms are candidates for pruning.

**Memory strategy**: We store accumulated norms on CPU rather than GPU to conserve VRAM, since we're processing the full model + calibration batches simultaneously.

In [ ]:
# Global storage for accumulated activation norms
_accumulated_act_norms = {}

def setup_mlp_hooks_for_importance(model, device):
    """
    Registers hooks on down_proj inputs (X_d) to calculate L2 norms
    for each neuron, following CFSP Equation 8.

    This function sets up the activation capture mechanism. For each MLP layer,
    we register a forward hook on down_proj that intercepts the input tensor
    (X_d) during forward passes, computes its L2 norm across batch and sequence
    dimensions, and accumulates these norms globally.

    The accumulated norms represent each neuron's total contribution across
    all calibration batches, providing a data-driven importance signal.

    Args:
        model: LlamaForCausalLM model to instrument with hooks
        device: Device where model is running (used for initialization)

    Returns:
        handles: List of hook handles (call handle.remove() to unregister)
    """
    global _accumulated_act_norms
    _accumulated_act_norms.clear()

    # Free memory before starting to avoid OOM errors during calibration
    gc.collect()
    torch.cuda.empty_cache()

    handles = []

    # Initialize storage on CPU to save VRAM during calibration
    # Each layer gets a tensor of shape [intermediate_size] initialized to zeros
    for idx, layer in enumerate(model.model.layers):
        intermediate_size = layer.mlp.down_proj.in_features
        _accumulated_act_norms[idx] = torch.zeros(
            intermediate_size,
            dtype=torch.float32,
            device='cpu'  # CPU storage prevents GPU memory overflow
        )

    def make_hook(layer_idx):
        """
        Factory function that creates a hook closure for a specific layer.
        Each hook captures X_d and accumulates its L2 norm.
        """
        def hook(module, input, output):
            """
            Hook function called automatically during forward pass.

            Captures X_d (input to down_proj) and calculates its L2 norm.
            X_d shape: [batch_size, seq_len, intermediate_size]
            Output: [intermediate_size] with ||X_d^i|| for each neuron i

            The L2 norm is computed as: ||X_d^i|| = sqrt(sum((X_d[:,:,i])^2))
            summed over batch and sequence dimensions.
            """
            X_d = input[0].detach()  # [B, S, I] - input to down_proj

            # Calculate L2 norm across batch and sequence dimensions
            # This gives us a single importance score per neuron
            act_norms_L2 = torch.norm(
                X_d.to(torch.float32),
                p=2,
                dim=(0, 1)  # Sum over batch (0) and sequence (1) dimensions
            )

            # Accumulate on CPU to save VRAM - we'll use these later for pruning
            _accumulated_act_norms[layer_idx] += act_norms_L2.cpu()

        return hook

    # Register hooks on every MLP layer's down_proj
    for idx, layer in enumerate(model.model.layers):
        handle = layer.mlp.down_proj.register_forward_hook(
            make_hook(idx)
        )
        handles.append(handle)

    print(f"✓ Registered {len(handles)} hooks on down_proj")

    return handles

def get_activation_norms():
    """
    Returns the accumulated L2 norms in a format ready for pruning.

    After running calibration batches through the model with hooks enabled,
    this function retrieves the accumulated activation norms for each layer.

    Returns:
        Dict[int, torch.Tensor]: {layer_idx: norms_L2 [intermediate_size]}
            Each tensor contains importance scores for all neurons in that layer.
    """
    return {
        layer_idx: norms.clone()
        for layer_idx, norms in _accumulated_act_norms.items()
    }

## Hybrid Importance Scoring

The key innovation: we combine **static** and **dynamic** signals to make pruning decisions.

**Static component**: Weight magnitude norms
- `||W_gate||`, `||W_up||`, `||W_down||` measure potential neuron capacity
- Large weights suggest a neuron *could* have significant impact
- Fast to compute, but ignores actual runtime behavior

**Dynamic component**: Activation norms from calibration data
- `||X_d||` measures actual neuron contributions on real inputs
- Captures which neurons *actually fire* for a given domain
- Expensive to compute, but provides ground truth about neuron utility

**Why both matter**: A neuron might have large weights but rarely activate (static overestimates importance), or have small weights but activate consistently (static underestimates importance). By multiplying the normalized static score by the activation norm, we get a hybrid metric that favors neurons that are both structurally significant *and* behaviorally active.

**Normalization strategy**: We scale gate, up, and down norms to [0,1] to equalize their contributions, preventing any single component from dominating the importance score.

In [ ]:
def compute_neuron_pair_importance(gate_weight, up_weight, down_weight, X_d_norm):
    """
    Hybrid CFSP-inspired importance: Static magnitude + Dynamic activation.

    Combines weight magnitudes (structural capacity) with activation norms
    (runtime behavior) to produce a single importance score per neuron.

    Args:
        gate_weight: Weight tensor from gate_proj [intermediate_size, hidden_size]
        up_weight: Weight tensor from up_proj [intermediate_size, hidden_size]
        down_weight: Weight tensor from down_proj [hidden_size, intermediate_size]
        X_d_norm: Accumulated activation norms [intermediate_size]

    Returns:
        importance_scores: Hybrid importance scores [intermediate_size]
    """
    # Convert all tensors to float32 for numerical stability
    gate_weight = gate_weight.float()
    up_weight = up_weight.float()
    down_weight = down_weight.float()
    X_d_norm = X_d_norm.float().to(gate_weight.device)

    # Static component: Compute L2 norms of weight vectors for each neuron
    # gate_proj and up_proj: each neuron is a row, so norm over dim=1
    gate_score = torch.norm(gate_weight, p=2, dim=1)
    up_score = torch.norm(up_weight, p=2, dim=1)
    
    # down_proj: each neuron is a column, so norm over dim=0
    down_score = torch.norm(down_weight, p=2, dim=0)

    # Normalize each component to [0, 1] range to equalize their scales
    # This prevents any single component from dominating the importance score
    gate_norm = gate_score / (gate_score.max() + 1e-8)
    up_norm = up_score / (up_score.max() + 1e-8)
    down_norm = down_score / (down_score.max() + 1e-8)

    # Combine the three static components
    # down_proj receives equal weight as it's the bottleneck where information contracts
    # Alternative weighting: 0.4 * down_norm + 0.4 * gate_norm + 0.2 * up_norm
    # Current approach: equal weighting for simplicity
    structural_score = down_norm + gate_norm + up_norm

    # Dynamic fusion: Multiply structural score by actual activation norms
    # This ensures neurons that are both structurally significant AND
    # active on calibration data receive high importance scores
    importance_scores = structural_score * X_d_norm

    return importance_scores

## Neuron Pair Pruning

With importance scores calculated, we now perform the actual surgery on the MLP layers.

**Key insight**: In GLU architectures, gate_proj and up_proj work as a **neuron pair**—they operate on the same input and their outputs are combined via element-wise multiplication. Therefore, we must prune corresponding neurons from both projections simultaneously.

The pruning process:
1. **Rank neurons** by importance scores (lowest to highest)
2. **Select top-k** most important neurons to keep
3. **Create new smaller layers** with dimensions:
   - gate_proj: [hidden_size → k] instead of [hidden_size → intermediate_size]
   - up_proj: [hidden_size → k]
   - down_proj: [k → hidden_size] instead of [intermediate_size → hidden_size]
4. **Copy weights** for only the kept neurons

**Dimension mapping**:
- gate_proj and up_proj: Neurons are **rows** (index with `weight[neuron_idx, :]`)
- down_proj: Neurons are **columns** (index with `weight[:, neuron_idx]`)

This ensures the pruned MLP maintains compatibility with the rest of the model—the input and output dimensions remain unchanged at `hidden_size`.

In [ ]:
def prune_neuron_pairs(mlp, prune_percent, X_d_norm, layer_idx):
    """
    Prunes neuron pairs from MLP block using hybrid importance scores.

    Reduces dimensions of gate_proj, up_proj, and down_proj layers by removing
    the least important neurons based on the combined static (weight magnitude)
    and dynamic (activation norm) scoring.

    Args:
        mlp: LlamaMLP module to prune
        prune_percent: Fraction of neurons to remove (e.g., 0.2 for 20%)
        X_d_norm: Tensor [intermediate_size] with accumulated L2 activation norms
        layer_idx: Layer index (for logging/debugging)

    Returns:
        new_gate_proj: Pruned gate projection layer
        new_up_proj: Pruned up projection layer
        new_down_proj: Pruned down projection layer
        k: New intermediate size (number of neurons kept)
    """

    # Extract weights from the three MLP projection layers
    gate_weight = mlp.gate_proj.weight.data
    up_weight = mlp.up_proj.weight.data
    down_weight = mlp.down_proj.weight.data

    original_intermediate_size = gate_weight.size(0)

    # Compute hybrid importance scores (static + dynamic)
    importance_scores = compute_neuron_pair_importance(
        gate_weight=gate_weight,
        up_weight=up_weight,
        down_weight=down_weight,
        X_d_norm=X_d_norm
    )

    # Calculate how many neurons to keep (k)
    # Ensure we keep at least 1 neuron to avoid zero-dimension tensors
    num_to_prune = min(
        int(prune_percent * original_intermediate_size),
        original_intermediate_size - 1
    )
    k = original_intermediate_size - num_to_prune

    if k <= 0:
        raise ValueError(f"Invalid k={k} for layer {layer_idx}")

    # Select top-k most important neurons to keep
    # topk returns (values, indices) - we only need indices
    _, indices_to_keep = torch.topk(
        importance_scores,
        k,
        largest=True,  # Keep highest importance scores
        sorted=True
    )

    # Sort indices to maintain neuron ordering (optional but cleaner)
    # This ensures weight matrices remain organized even after pruning
    indices_to_keep = indices_to_keep.sort().values

    # Create new pruned linear layers with reduced dimensions
    new_gate_proj = nn.Linear(
        mlp.gate_proj.in_features,  # Input: hidden_size (unchanged)
        k,                          # Output: reduced intermediate_size
        bias=False
    ).to(device)

    new_up_proj = nn.Linear(
        mlp.up_proj.in_features,    # Input: hidden_size (unchanged)
        k,                          # Output: reduced intermediate_size
        bias=False
    ).to(device)

    new_down_proj = nn.Linear(
        k,                          # Input: reduced intermediate_size
        mlp.down_proj.out_features, # Output: hidden_size (unchanged)
        bias=False
    ).to(device)

    # Copy weights for kept neurons only
    # gate_proj and up_proj: neurons are rows, so index first dimension
    new_gate_proj.weight.data = gate_weight[indices_to_keep, :]
    new_up_proj.weight.data = up_weight[indices_to_keep, :]
    
    # down_proj: neurons are columns, so index second dimension
    new_down_proj.weight.data = down_weight[:, indices_to_keep]

    return new_gate_proj, new_up_proj, new_down_proj, k

In [22]:
def update_model(model, prune_percent, activation_norms):
    """
    Applies pruning to all MLP layers in the model.

    Args:
        model: LlamaForCausalLM model to prune
        prune_percent: Fraction of neurons to remove
        activation_norms: Dict mapping layer_idx -> X_d_norm tensor

    Returns:
        model: Pruned model with updated configuration
    """

    new_intermediate_size = None
    pruning_stats = []

    print(f"\n{'='*60}")
    print(f"Starting pruning with {prune_percent*100:.1f}% width pruning")
    print(f"{'='*60}\n")

    for idx, layer in enumerate(model.model.layers):
        mlp = layer.mlp

        if idx not in activation_norms:
            raise KeyError(f"No activation norms for layer {idx}")

        X_d_norm = activation_norms[idx]
        original_size = mlp.gate_proj.out_features

        # Prune neuron pairs
        new_gate_proj, new_up_proj, new_down_proj, new_size = prune_neuron_pairs(
            mlp=mlp,
            prune_percent=prune_percent,
            X_d_norm=X_d_norm,
            layer_idx=idx
        )

        # Replace layers
        mlp.gate_proj = new_gate_proj
        mlp.up_proj = new_up_proj
        mlp.down_proj = new_down_proj

        pruning_stats.append({
            'layer': idx,
            'original_size': original_size,
            'new_size': new_size,
            'pruned': original_size - new_size,
            'kept_percent': (new_size / original_size) * 100
        })

        if new_intermediate_size is None:
            new_intermediate_size = new_size

        if (idx + 1) % 4 == 0:
            print(f"  Pruned layers {idx-3:2d}-{idx:2d}: "
                  f"{original_size} → {new_size} neurons "
                  f"({(new_size/original_size)*100:.1f}% kept)")

    # Update model configuration
    model.config.intermediate_size = new_intermediate_size

    print(f"\n{'='*60}")
    print(f"Pruning completed!")
    print(f"{'='*60}")
    print(f"  Layers pruned: {len(pruning_stats)}")
    print(f"  Original intermediate size: {original_size}")
    print(f"  New intermediate size: {new_intermediate_size}")
    print(f"  Neurons pruned per layer: {original_size - new_intermediate_size}")
    print(f"  Effective width pruning: {((original_size - new_intermediate_size) / original_size) * 100:.2f}%")
    print(f"{'='*60}\n")

    return model

# Calibration and Pruning

Now we'll create two pruned models using different calibration datasets:
1. **Wiki-pruned**: Calibrated on WikiText-2
2. **SMS-pruned**: Calibrated on SMS Spam

## Wiki-Calibrated Model

In [23]:
print("="*60)
print("WIKI CALIBRATION")
print("="*60)

# Step 1: Setup hooks
print("\nSetting up activation hooks...")
handles_wiki = setup_mlp_hooks_for_importance(model, device)

# Step 2: Run calibration forward passes
print("\nRunning calibration forward passes on WikiText...")
model.eval()

with torch.no_grad():
    for batch_idx, batch in enumerate(tqdm(dataloaderwiki, desc="Wiki Calibration")):
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }
        outputs = model(**inputs)

        if (batch_idx + 1) % 10 == 0:
            torch.cuda.empty_cache()

print(f"\n✓ Processed {len(dataloaderwiki)} batches")

# Step 3: Clean up hooks
print("Removing hooks...")
for handle in handles_wiki:
    handle.remove()

# Step 4: Get activation norms
print("Extracting activation statistics...")
activation_norms_wiki = get_activation_norms()
print(f"✓ Collected activation norms for {len(activation_norms_wiki)} layers")

WIKI CALIBRATION

Setting up activation hooks...
✓ Registered 16 hooks on down_proj

Running calibration forward passes on WikiText...


Wiki Calibration: 100%|██████████| 25/25 [00:15<00:00,  1.59it/s]


✓ Processed 25 batches
Removing hooks...
Extracting activation statistics...
✓ Collected activation norms for 16 layers


In [24]:
# Prune the model using Wiki activations
wiki_model = update_model(copy.deepcopy(model), PRUNE_PERCENT, activation_norms_wiki)


Starting pruning with 20.0% width pruning

  Pruned layers  0- 3: 8192 → 6554 neurons (80.0% kept)
  Pruned layers  4- 7: 8192 → 6554 neurons (80.0% kept)
  Pruned layers  8-11: 8192 → 6554 neurons (80.0% kept)
  Pruned layers 12-15: 8192 → 6554 neurons (80.0% kept)

Pruning completed!
  Layers pruned: 16
  Original intermediate size: 8192
  New intermediate size: 6554
  Neurons pruned per layer: 1638
  Effective width pruning: 20.00%



In [25]:
del(model)
clear_gpu_cache()

In [26]:
# Test wiki model
generated_wiki = generate_text(wiki_model, tokenizer, prompt, device)
print(f"Wiki-pruned model: {generated_wiki}")

Wiki-pruned model: Paris is the capital of France and is located in the north-east of the country. The city has a population of 2.1 million people, making it the 3rd largest city in France. It is also the largest in terms of surface area, with 100


In [27]:
clear_gpu_cache()
metrics_wiki_wiki = evaluate_metrics(wiki_model, dataloaderwiki)


Evaluating: 100%|██████████| 25/25 [00:16<00:00,  1.50it/s]


In [28]:
metrics_wiki_sms = evaluate_metrics(wiki_model, dataloadersms)


Evaluating: 100%|██████████| 25/25 [00:16<00:00,  1.47it/s]


In [29]:
wiki_wiki_timing = measure_detailed_performance(wiki_model, tokenizer, dataloaderwiki, max_samples=10)

Measuring performance on 10 samples...


Performance test: 100%|██████████| 10/10 [00:39<00:00,  3.93s/it]


In [30]:
wiki_params = count_parameters(wiki_model)

In [31]:
wiki_sms_timing = measure_detailed_performance(wiki_model, tokenizer, dataloadersms, max_samples=10)

Measuring performance on 10 samples...


Performance test: 100%|██████████| 10/10 [00:38<00:00,  3.85s/it]


In [32]:
del(wiki_model)
clear_gpu_cache

<function utils.clear_gpu_cache()>

## SMS-Calibrated Model

In [33]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [34]:
# Clear GPU cache before SMS calibration
clear_gpu_cache()

print("="*60)
print("SMS CALIBRATION")
print("="*60)

# Step 1: Setup hooks
print("\nSetting up activation hooks...")
handles_sms = setup_mlp_hooks_for_importance(model, device)

# Step 2: Run calibration forward passes
print("\nRunning calibration forward passes on SMS...")
model.eval()

with torch.no_grad():
    for batch_idx, batch in enumerate(tqdm(dataloadersms, desc="SMS Calibration")):
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }
        outputs = model(**inputs)

        if (batch_idx + 1) % 10 == 0:
            torch.cuda.empty_cache()

print(f"\n✓ Processed {len(dataloadersms)} batches")

# Step 3: Clean up hooks
print("Removing hooks...")
for handle in handles_sms:
    handle.remove()

# Step 4: Get activation norms
print("Extracting activation statistics...")
activation_norms_sms = get_activation_norms()
print(f"✓ Collected activation norms for {len(activation_norms_sms)} layers")

SMS CALIBRATION

Setting up activation hooks...
✓ Registered 16 hooks on down_proj

Running calibration forward passes on SMS...


SMS Calibration: 100%|██████████| 25/25 [00:15<00:00,  1.61it/s]


✓ Processed 25 batches
Removing hooks...
Extracting activation statistics...
✓ Collected activation norms for 16 layers


In [35]:
# Prune the model using SMS activations
sms_model = update_model(copy.deepcopy(model), PRUNE_PERCENT, activation_norms_sms)


Starting pruning with 20.0% width pruning

  Pruned layers  0- 3: 8192 → 6554 neurons (80.0% kept)
  Pruned layers  4- 7: 8192 → 6554 neurons (80.0% kept)
  Pruned layers  8-11: 8192 → 6554 neurons (80.0% kept)
  Pruned layers 12-15: 8192 → 6554 neurons (80.0% kept)

Pruning completed!
  Layers pruned: 16
  Original intermediate size: 8192
  New intermediate size: 6554
  Neurons pruned per layer: 1638
  Effective width pruning: 20.00%



In [36]:
del(model)
clear_gpu_cache()

In [37]:
# Test SMS model
generated_sms = generate_text(sms_model, tokenizer, prompt, device)
print(f"SMS-pruned model: {generated_sms}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


SMS-pruned model: Paris is the capital of the French department of Paris. It is located on the Seine River, which flows through the city. The city has a population of 1.8 million people. Paris is a major city in the world. Its population is estimated to be 


In [38]:
metrics_sms_wiki = evaluate_metrics(sms_model, dataloaderwiki)
metrics_sms_sms = evaluate_metrics(sms_model, dataloadersms)

Evaluating: 100%|██████████| 25/25 [00:17<00:00,  1.45it/s]


In [39]:
sms_wiki_timing = measure_detailed_performance(sms_model, tokenizer, dataloaderwiki, max_samples=10)

Measuring performance on 10 samples...


Performance test:   0%|          | 0/10 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Performance test:  10%|█         | 1/10 [00:04<00:37,  4.19s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Performance test:  20%

In [40]:
sms_sms_timing = measure_detailed_performance(sms_model, tokenizer, dataloadersms, max_samples=10)

Measuring performance on 10 samples...


Performance test:   0%|          | 0/10 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Performance test:  10%|█         | 1/10 [00:04<00:37,  4.13s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Performance test:  20%

## Parameter Comparison

In [41]:
sms_params = count_parameters(sms_model)

print("\n" + "="*60)
print("PARAMETER COUNTS")
print("="*60)
print(f"Original model:     {original_params:,} parameters")
print(f"Wiki-pruned model:  {wiki_params:,} parameters ({((original_params - wiki_params) / original_params * 100):.2f}% reduction)")
print(f"SMS-pruned model:   {sms_params:,} parameters ({((original_params - sms_params) / original_params * 100):.2f}% reduction)")
print("="*60)


PARAMETER COUNTS
Original model:     1,235,814,400 parameters
Wiki-pruned model:  1,074,792,448 parameters (13.03% reduction)
SMS-pruned model:   1,074,792,448 parameters (13.03% reduction)


# Cross-Evaluation Matrix

Now we evaluate all three models (base, wiki-pruned, sms-pruned) on both datasets (Wiki and SMS) to answer our core research question: **Does calibration dataset selection influence pruning decisions, and do the resulting models specialize for their calibration domain?**

## The Experiment Design

**Models**:
1. Base model (unpruned Llama-3.2-1B)
2. Wiki-pruned (calibrated on WikiText-2, 20% width reduction)
3. SMS-pruned (calibrated on SMS Spam, 20% width reduction)

**Evaluation datasets**:
- WikiText-2 (formal, complex)
- SMS Spam (informal, simple)

**Hypothesis**:
- Wiki-pruned should perform **better on WikiText** than on SMS (domain match)
- SMS-pruned should perform **better on SMS** than on WikiText (domain match)
- Cross-domain performance (wiki-pruned on SMS, sms-pruned on Wiki) should be **worse** than domain-matched performance

This cross-evaluation matrix reveals whether data-driven pruning creates specialized models or if the calibration dataset choice doesn't significantly impact neuron selection.

We'll measure:
- **Perplexity**: Language modeling quality (lower is better)
- **Loss**: Cross-entropy loss (lower is better)
- **Inference speed**: Tokens per second (higher is better)

Let's see if our hypothesis holds!

In [42]:
# Clear original model to save memory
clear_gpu_cache()

## Loss and Perplexity Evaluation

In [43]:
print("\n" + "="*60)
print("EVALUATING ON WIKITEXT-2")
print("="*60)

print("\nBase model on Wiki:", metrics_base_wiki)
print("Wiki-pruned on Wiki:", metrics_wiki_wiki)
print("SMS-pruned on Wiki:", metrics_sms_wiki)


EVALUATING ON WIKITEXT-2

Base model on Wiki: {'loss': 3.2460288122350294, 'perplexity': np.float64(25.688124727046315)}
Wiki-pruned on Wiki: {'loss': 3.5913135137696126, 'perplexity': np.float64(36.28170115548473)}
SMS-pruned on Wiki: {'loss': 3.8846293323963836, 'perplexity': np.float64(48.64890654342502)}


In [44]:
print("\n" + "="*60)
print("EVALUATING ON SMS SPAM")
print("="*60)

print("\nBase model on SMS:", metrics_base_sms)
print("Wiki-pruned on SMS:", metrics_wiki_sms)
print("SMS-pruned on SMS:", metrics_sms_sms)


EVALUATING ON SMS SPAM

Base model on SMS: {'loss': 4.8114213257195955, 'perplexity': np.float64(122.90618314977401)}
Wiki-pruned on SMS: {'loss': 5.206973014926048, 'perplexity': np.float64(182.540673179065)}
SMS-pruned on SMS: {'loss': 5.109189337836538, 'perplexity': np.float64(165.53610660777312)}


## Text Generation Comparison

## Performance Measurement (Inference Speed)

In [45]:
print("\n" + "="*60)
print("PERFORMANCE MEASUREMENT ON WIKI")
print("="*60)


print(f"\nBase model:       {base_wiki_timing['avg_time_sec']:.4f}s ({base_wiki_timing['tokens_per_sec']:.2f} tok/s)")
print(f"Wiki-pruned:      {wiki_wiki_timing['avg_time_sec']:.4f}s ({wiki_wiki_timing['tokens_per_sec']:.2f} tok/s)")
print(f"SMS-pruned:       {sms_wiki_timing['avg_time_sec']:.4f}s ({sms_wiki_timing['tokens_per_sec']:.2f} tok/s)")


PERFORMANCE MEASUREMENT ON WIKI

Base model:       0.6791s (29.74 tok/s)
Wiki-pruned:      1.3088s (35.45 tok/s)
SMS-pruned:       1.2069s (34.88 tok/s)


In [46]:
print("\n" + "="*60)
print("PERFORMANCE MEASUREMENT ON SMS")
print("="*60)



print(f"\nBase model:       {base_sms_timing['avg_time_sec']:.4f}s ({base_sms_timing['tokens_per_sec']:.2f} tok/s)")
print(f"Wiki-pruned:      {wiki_sms_timing['avg_time_sec']:.4f}s ({wiki_sms_timing['tokens_per_sec']:.2f} tok/s)")
print(f"SMS-pruned:       {sms_sms_timing['avg_time_sec']:.4f}s ({sms_sms_timing['tokens_per_sec']:.2f} tok/s)")


PERFORMANCE MEASUREMENT ON SMS

Base model:       1.0957s (34.32 tok/s)
Wiki-pruned:      1.2819s (35.88 tok/s)
SMS-pruned:       1.3891s (35.99 tok/s)


# Summary and Results Analysis

In [47]:
# Create comprehensive results table
results = {
    'Model': ['Base', 'Wiki-pruned', 'SMS-pruned'],
    'Parameters': [original_params, wiki_params, sms_params],
    'Param Reduction %': [
        0,
        ((original_params - wiki_params) / original_params * 100),
        ((original_params - sms_params) / original_params * 100)
    ],
    'PPL Wiki': [
        metrics_base_wiki['perplexity'],
        metrics_wiki_wiki['perplexity'],
        metrics_sms_wiki['perplexity']
    ],
    'PPL SMS': [
        metrics_base_sms['perplexity'],
        metrics_wiki_sms['perplexity'],
        metrics_sms_sms['perplexity']
    ],
    'Loss Wiki': [
        metrics_base_wiki['loss'],
        metrics_wiki_wiki['loss'],
        metrics_sms_wiki['loss']
    ],
    'Loss SMS': [
        metrics_base_sms['loss'],
        metrics_wiki_sms['loss'],
        metrics_sms_sms['loss']
    ],
    'Time Wiki (s)': [
        base_wiki_timing['avg_time_sec'],
        wiki_wiki_timing['avg_time_sec'],
        sms_wiki_timing['avg_time_sec']
    ],
    'Time SMS (s)': [
        base_sms_timing['avg_time_sec'],
        wiki_sms_timing['avg_time_sec'],
        sms_sms_timing['avg_time_sec']
    ],
    'Tok/s Wiki': [
        base_wiki_timing['tokens_per_sec'],
        wiki_wiki_timing['tokens_per_sec'],
        sms_wiki_timing['tokens_per_sec']
    ],
    'Tok/s SMS': [
        base_sms_timing['tokens_per_sec'],
        wiki_sms_timing['tokens_per_sec'],
        sms_sms_timing['tokens_per_sec']
    ],
}

df_results = pd.DataFrame(results)

print("\n" + "="*80)
print("COMPREHENSIVE RESULTS")
print("="*80)
print(df_results.to_string(index=False))
print("="*80)


COMPREHENSIVE RESULTS
      Model  Parameters  Param Reduction %  PPL Wiki    PPL SMS  Loss Wiki  Loss SMS  Time Wiki (s)  Time SMS (s)  Tok/s Wiki  Tok/s SMS
       Base  1235814400           0.000000 25.688125 122.906183   3.246029  4.811421       0.679144      1.095701   29.743339  34.315922
Wiki-pruned  1074792448          13.029623 36.281701 182.540673   3.591314  5.206973       1.308820      1.281878   35.451768  35.884851
 SMS-pruned  1074792448          13.029623 48.648907 165.536107   3.884629  5.109189       1.206911      1.389136   34.882451  35.993593


# Results Visualization

Let's create visual comparisons to better understand the performance differences across models and datasets. These charts will make it easier to spot patterns in perplexity degradation, inference speed changes, and domain specialization effects.

## Perplexity Comparison

In [ ]:
# Perplexity comparison across all models and datasets
models = ['Base', 'Wiki-pruned', 'SMS-pruned']
datasets = ['WikiText', 'SMS']

# Extract perplexity values from results
perplexity_data = [
    [metrics_base_wiki['perplexity'], metrics_base_sms['perplexity']],
    [metrics_wiki_wiki['perplexity'], metrics_wiki_sms['perplexity']],
    [metrics_sms_wiki['perplexity'], metrics_sms_sms['perplexity']]
]

fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(models))
width = 0.35

bars1 = ax.bar(x - width/2, [row[0] for row in perplexity_data],
               width, label='WikiText', alpha=0.8, color='skyblue')
bars2 = ax.bar(x + width/2, [row[1] for row in perplexity_data],
               width, label='SMS', alpha=0.8, color='lightcoral')

ax.set_ylabel('Perplexity (Lower is Better)', fontsize=12)
ax.set_title('Perplexity Comparison: 3 Models × 2 Datasets', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{height:.1f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## Performance Metrics Comparison

In [ ]:
# Inference speed comparison (tokens per second)
wiki_speed = [base_wiki_timing['tokens_per_sec'], 
              wiki_wiki_timing['tokens_per_sec'], 
              sms_wiki_timing['tokens_per_sec']]
sms_speed = [base_sms_timing['tokens_per_sec'], 
             wiki_sms_timing['tokens_per_sec'], 
             sms_sms_timing['tokens_per_sec']]

fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(models))
width = 0.35

bars1 = ax.bar(x - width/2, wiki_speed, width, label='WikiText', alpha=0.8, color='forestgreen')
bars2 = ax.bar(x + width/2, sms_speed, width, label='SMS', alpha=0.8, color='orange')

ax.set_ylabel('Tokens per Second (Higher is Better)', fontsize=12)
ax.set_title('Inference Speed Comparison: 3 Models × 2 Datasets', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{height:.1f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## Loss Degradation Analysis

In [ ]:
# Loss degradation across models
wiki_loss = [metrics_base_wiki['loss'], 
             metrics_wiki_wiki['loss'], 
             metrics_sms_wiki['loss']]
sms_loss = [metrics_base_sms['loss'], 
            metrics_wiki_sms['loss'], 
            metrics_sms_sms['loss']]

fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(models))

ax.plot(x, wiki_loss, marker='o', linewidth=2, markersize=8,
        label='WikiText', color='blue')
ax.plot(x, sms_loss, marker='s', linewidth=2, markersize=8,
        label='SMS', color='red')

ax.set_ylabel('Loss (Lower is Better)', fontsize=12)
ax.set_xlabel('Model', fontsize=12)
ax.set_title('Loss Degradation Across Models', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.grid(True, alpha=0.3)

# Add value labels
for i, (wl, sl) in enumerate(zip(wiki_loss, sms_loss)):
    ax.text(i, wl, f'{wl:.3f}', ha='center', va='bottom', fontsize=9, color='blue')
    ax.text(i, sl, f'{sl:.3f}', ha='center', va='bottom', fontsize=9, color='red')

plt.tight_layout()
plt.show()

# Summary and Analysis

This notebook explored data-driven neuron pruning using activation patterns from calibration datasets. We created two differently-pruned models (Wiki-calibrated and SMS-calibrated) and cross-evaluated them to test whether calibration dataset selection influences pruning decisions and model specialization.

## Key Findings

### Parameter Reduction
- **Both pruned models**: 13.03% parameter reduction (161M parameters removed)
- **Target pruning**: 20% width reduction (8,192 → 6,554 neurons per layer)
- **Actual savings**: Modest compared to the 26% achieved with 40% pruning in static method (NB01)

### Perplexity Impact

The results reveal **strong domain sensitivity**:

**WikiText evaluation**:
- Base model: 25.69 PPL
- Wiki-pruned: 36.28 PPL (+41.2% increase)
- SMS-pruned: 48.65 PPL (+89.3% increase) ⚠️

**SMS evaluation**:
- Base model: 122.91 PPL
- Wiki-pruned: 182.54 PPL (+48.5% increase) ⚠️
- SMS-pruned: 165.54 PPL (+34.7% increase)

**Critical observation**: SMS-pruned performs better on SMS than Wiki-pruned (165.5 vs 182.5 PPL), providing evidence of **domain specialization**. However, both pruned models suffer severe degradation in cross-domain evaluation.

### Loss Analysis

Loss follows similar patterns to perplexity:
- Wiki-pruned excels on WikiText (3.59 vs 3.88 for SMS-pruned)
- SMS-pruned excels on SMS (5.11 vs 5.21 for Wiki-pruned)
- Domain mismatch causes significant loss increases

This confirms that calibration dataset choice **does influence** which neurons get pruned, creating models that are partially specialized for their calibration domain.

## Performance Trade-offs

### Inference Speed: An Unexpected Finding

**Hypothesis**: Pruned models should be faster (fewer parameters → less computation)

**Reality**: Pruned models show **marginal or even slower** performance in some cases:

**WikiText**:
- Base: 29.74 tok/s
- Wiki-pruned: 35.45 tok/s (+19% faster)
- SMS-pruned: 34.88 tok/s (+17% faster)

**SMS**:
- Base: 34.32 tok/s
- Wiki-pruned: 35.88 tok/s (+4.5% faster)
- SMS-pruned: 35.99 tok/s (+4.9% faster)

**Why are gains minimal?**
1. **Memory bandwidth bottleneck**: Modern GPUs are often memory-bound, not compute-bound
2. **Irregular tensor shapes**: Non-standard intermediate sizes (6,554) may not map efficiently to GPU hardware
3. **Small model size**: Llama-3.2-1B already fits comfortably in memory; pruning doesn't alleviate a bottleneck
4. **Framework overhead**: PyTorch may not fully optimize for irregular layer dimensions

**Conclusion**: 13% parameter reduction yields only 5-19% speed improvement—insufficient to justify the severe perplexity degradation.

## Domain Specialization Analysis

**Does data-driven pruning create specialized models?**

**Evidence for specialization**:
✅ SMS-pruned outperforms Wiki-pruned on SMS dataset (165.5 vs 182.5 PPL)  
✅ Wiki-pruned outperforms SMS-pruned on WikiText (36.3 vs 48.6 PPL)  
✅ Each model performs best on its calibration domain

**Evidence against strong specialization**:
⚠️ Both pruned models degrade significantly on both datasets  
⚠️ Cross-domain transfer is poor (Wiki-pruned on SMS: +48.5% PPL)  
⚠️ Specialization gains are modest compared to overall quality loss

**Interpretation**: Calibration datasets **do** influence which neurons survive pruning, creating measurable domain preferences. However, the specialization is **weak**—not strong enough to make domain-specific pruning a clear win over general-purpose compression.

## Comparison to Static Pruning (Notebook 01)

| Metric | Static Pruning (NB01) | Data-Driven (NB02) |
|--------|----------------------|-------------------|
| **Width reduction** | 40% | 20% |
| **Parameter savings** | 26.06% | 13.03% |
| **Method** | Weight magnitude only | Magnitude + activations |
| **Calibration required** | No | Yes (100 samples) |
| **Domain sensitivity** | None | High |
| **Quality degradation** | Severe (PPL +386-1471%) | Severe (PPL +34-89%) |

**Key insight**: Despite using activation patterns from real data, data-driven pruning with 20% width reduction produces similar quality degradation to static pruning with 40% reduction. This suggests:

1. **Activation-aware pruning is more conservative**: It preserves more neurons (20% vs 40% pruning)
2. **Calibration dataset matters more than expected**: Different datasets produce different models
3. **Hybrid scoring helps, but not dramatically**: The improvement from adding dynamic signals is modest

## Practical Implications

**When to use data-driven pruning**:
✅ You have a **specific deployment domain** with representative calibration data  
✅ You can afford **domain-specific model variants** (e.g., wiki-model for documents, sms-model for chat)  
✅ You need to **minimize quality loss** in a target domain  
✅ You want to **understand which neurons matter** for a specific task

**When to avoid it**:
❌ You need a **general-purpose** compressed model  
❌ Cross-domain performance is important  
❌ You lack **representative calibration data**  
❌ Speed/size improvements don't justify quality degradation  
❌ You want simplicity (static pruning requires no calibration)

**Bottom line**: Data-driven pruning creates domain-specialized models, but the specialization is **weak** and comes at the cost of **severe cross-domain degradation**. For most applications, the trade-off is unfavorable.

## Open Questions and Future Work

1. **Longer calibration**: Would using 1,000+ samples instead of 100 improve pruning quality?

2. **Hybrid calibration**: Can we combine WikiText + SMS during calibration to create a more robust pruned model?

3. **Layer-specific pruning rates**: Should early layers (general features) be pruned less aggressively than late layers (task-specific)?

4. **Fine-tuning after pruning**: Would brief fine-tuning on the calibration dataset recover lost performance?

5. **Alternative importance metrics**: Are there better ways to combine static and dynamic signals (e.g., attention to activation correlation)?

6. **Hardware-aware pruning**: Can we constrain intermediate sizes to GPU-friendly dimensions (powers of 2) to maximize speed gains?

7. **Structured vs unstructured**: How does neuron-level pruning compare to more fine-grained weight pruning?

## Conclusion

This notebook demonstrated that **calibration dataset selection significantly influences data-driven pruning decisions**, creating models with measurable domain preferences. However, the resulting specialization is **insufficient to justify the severe quality degradation** in most practical scenarios.

**Key takeaways**:
- ✅ Data-driven pruning **does** create domain-specialized models
- ⚠️ Specialization is **weak** compared to overall quality loss
- ❌ Speed gains are **minimal** (5-19%) despite 13% parameter reduction
- 📊 Cross-domain performance is **severely degraded**

For general-purpose model compression, **alternative techniques** (quantization, distillation, layer dropping) may offer better trade-offs. Data-driven width pruning is best suited for **highly specialized, domain-specific deployments** where calibration data perfectly matches the target distribution.

In [48]:
print(generated_base)
print(generated_wiki)
print(generated_sms)

Paris is the capital of France and the largest city in the country. It is located on the River Seine and is one of the most popular tourist destinations in Europe. The city has a rich history and culture, and it is home to many famous landmarks, including the E
Paris is the capital of France and is located in the north-east of the country. The city has a population of 2.1 million people, making it the 3rd largest city in France. It is also the largest in terms of surface area, with 100
Paris is the capital of the French department of Paris. It is located on the Seine River, which flows through the city. The city has a population of 1.8 million people. Paris is a major city in the world. Its population is estimated to be 


40% pruning expansion rate.
```
================================================================================
COMPREHENSIVE RESULTS
================================================================================
      Model  Parameters  Param Reduction %  PPL Wiki    PPL SMS  Loss Wiki  Loss SMS  Time Wiki (s)  Time SMS (s)  Tok/s Wiki  Tok/s SMS
       Base  1235814400           0.000000 25.688125 122.906183   3.246029  4.811421       0.679144      1.095701   29.743339  34.315922
Wiki-pruned  1074792448          13.029623 36.281701 182.540673   3.591314  5.206973       1.308820      1.281878   35.451768  35.884851
 SMS-pruned  1074792448          13.029623 48.648907 165.536107   3.884629  5.109189       1.206911      1.389136   34.882451  35.993593
================================================================================
````
